In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import nltk
import random
nltk.download('punkt')

In [ ]:
file = pd.read_csv("../input/515k-hotel-reviews-data-in-europe/Hotel_Reviews.csv")
file.head()

In [ ]:

new_file_ = file.drop(["Average_Score","Additional_Number_of_Scoring","Review_Date","Hotel_Name","Reviewer_Nationality","Total_Number_of_Reviews","Total_Number_of_Reviews_Reviewer_Has_Given","Tags","days_since_review","lat","lng" ],axis = 1)
new_file = new_file_.loc[0:20000]
test_file = new_file_.loc[0:20500]

random_size = len(new_file)/4


print(len(new_file))

In [ ]:
data = []
data_test = []
all_text = ""
word2idx = {}
idx2word = {}


key = 0

word2idx.update({"<end>":key})
idx2word.update({key:"<end>"})

key = 1

max_ = 0
size = [] 

print(len(test_file))

#combine negative and positive data

for i in range(0,len(test_file)):
    if(i < len(new_file)):
        size = nltk.word_tokenize(new_file["Negative_Review"][i] + new_file["Positive_Review"][i])
        data.append(( nltk.word_tokenize(new_file["Negative_Review"][i] + new_file["Positive_Review"][i]), round(new_file["Reviewer_Score"][i])))
    else:
        data_test.append(( nltk.word_tokenize(test_file["Negative_Review"][i] + test_file["Positive_Review"][i]), round(test_file["Reviewer_Score"][i])))
        size =  nltk.word_tokenize(test_file["Negative_Review"][i] + test_file["Positive_Review"][i])
        
    all_text += test_file["Negative_Review"][i] + test_file["Positive_Review"][i]
    
    if(max_ < len(size)):
        max_ = len(size)
    

text = nltk.word_tokenize(all_text)


#create a vocab dictionary
for index, word in enumerate(text):
    if word not in word2idx:
        word2idx.update({word:key})
        idx2word.update({key:word})
        key += 1
             
print(max_)

vocab_size = len(word2idx)
embbed_size = 10



In [ ]:
negative = 0
positive = 0

for key, rev in enumerate(data):            
    if(rev[1] < 5):
        negative += 1
    else:
        rand = random.randrange(0,int(random_size))
        if rand != 1:
            data.pop(key)
            
for key, rev in enumerate(data):
    if(rev[1] < 5):
        negative += 1
    
    else:
        positive += 1
        
 
print(negative)
print(positive)

print(len(data))
ratio = negative/len(data) * 100
print(ratio)

In [ ]:
class classifier(nn.Module):
    
    def __init__(self,vocab_size, embbed_size):
        super(classifier, self).__init__()
        
        self.vocab_size = vocab_size
        self.embbed_size = embbed_size
        
        self.embbed = nn.Embedding(vocab_size, embbed_size)
        self.rnn = nn.LSTM(embbed_size,10*embbed_size,2,True,False,0.2,True)
        self.linear1 = nn.Linear(2*10*embbed_size*max_, 10)
        self.softmax = nn.LogSoftmax(dim=1)
       
    def forward(self,input_):
        
        embeded_input = self.embbed(input_)
        embeded_input = torch.unsqueeze(embeded_input, dim = 1)
        output, (h,m) = self.rnn(embeded_input)
        in_ = output.view((1,-1))
        
        out = self.linear1(in_)
        out = self.softmax(out)
        
        return out 

        
        

In [ ]:
model = classifier(vocab_size, embbed_size)
optimizer = optim.Adam(model.parameters())
loss_function  = nn.CrossEntropyLoss()

In [ ]:
def accuracy():
    
    model.eval()

    with torch.no_grad():
        
        generous_correct  = 0
        correct  = 0
        for key, review in enumerate(data_test):

            try:
                input_ = torch.zeros(max_,dtype = int)

                for index, i in enumerate(review[0]):
                    input_[index] = word2idx[i]     

                target =  torch.tensor([review[1] -1])

                out = model(input_)

                result = torch.argmax(out, dim = 1) 


                if(target == result):
                    generous_correct += 1
                    correct += 1
                elif(target == result + 1):
                    generous_correct += 1
                elif(target == result - 1):
                    generous_correct += 1

            except:

                continue 

        ratio = (correct/len(data_test)) * 100
        ratio_ = (generous_correct/len(data_test)) * 100

        print("correct ", ratio)
        print("error correct ", ratio_)
        
        model.train()


In [ ]:
correct = 0
loss_value  = 0

for j in range(5):
    
    print("epoch ", j+1)
    running_loss = 0
    
    for key, review in enumerate(data):
        
        model.zero_grad()
        
        try:
            input_ = torch.zeros(max_, dtype = int)

            for index, i in enumerate(review[0]):
                input_[index] = word2idx[i]   

            target =  torch.tensor([review[1] -1])
            
            out = model(input_)
            loss = loss_function(out, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
                
            if key % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{j + 1}, {key + 1:5d}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
                accuracy()


        except:
            
            print("word not in dictionary")
            continue 
        



In [ ]:
accuracy()

In [ ]:
PATH = "state_dict_model.pt"
torch.save(model.state_dict(), PATH)